Temporary results of Corporate Incom Tax distribution to tax-payers

In [1]:
import sys
import taxcalc
from taxcalc import *
import pandas as pd
import numpy as np
from numba import jit
import numba

In [2]:
records_x = Records("puf.csv")
records_y = Records("puf.csv")

In [3]:
policy_x = Policy()
policy_y = Policy()
behavior_y = Behavior()
calcX = Calculator(policy_x, records_x)
calcY = Calculator(policy_y, records_y, behavior=behavior_y)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [4]:
calcX.advance_to_year(2017)
calcY.advance_to_year(2017)
assert calcX.current_year == 2017
assert calcY.current_year == 2017

In [5]:
calcX.calc_all()

In [6]:
reform = {2017: {"_II_rt7": [0.496]}}
policy_y.implement_reform(reform)
# Demonstrate that Plan X and Plan Y calculators are indeed different. 
print(calcX.policy.II_rt7)
print(calcY.policy.II_rt7)

0.396
0.496


In [7]:
behavior_y.update_behavior({2017: {'_BE_inc': [-0.15], '_BE_sub': [0.4]}})
calcY_behavior1 = Behavior.response(calcX, calcY)
behavior_y.update_behavior({2017: {'_BE_inc': [-0.15], '_BE_sub': [0.5]}})
calcY_behavior2 = Behavior.response(calcX, calcY)
behavior_y.update_behavior({2017: {'_BE_inc': [0.0], '_BE_sub': [0.4]}})
calcY_behavior3 = Behavior.response(calcX, calcY)

In [8]:
# Demonstrate that taxpayers' income was affected by the behavioral effects.
print(calcY.records.e00200.sum())
print(calcY_behavior1.records.e00200.sum())
print(calcY_behavior2.records.e00200.sum())
print(calcY_behavior3.records.e00200.sum())

40681683225.0
38887827999.4
38373606744.6
38624798205.8


In [9]:
# Demonstrate that total tax was affected by the behavioral effects. 
print(calcX.records.combined.sum())
print(calcY.records.combined.sum())
print(calcY_behavior1.records.combined.sum())
print(calcY_behavior2.records.combined.sum())
print(calcY_behavior3.records.combined.sum())

43476646184.5
46402047946.2
43849701719.0
43185112636.7
43604367079.6


In [10]:
DistCorpIncTax(calcX.policy, calcX.records)

,share_corptax_burden
0,7.964186e+00
1,1.887646e+07
2,3.708569e+07
3,-2.695613e+05
4,-4.994644e+04
5,8.511275e+05
6,6.965848e+06
7,1.191294e+06
8,6.846493e+05
9,1.054586e+06


In [11]:
create_corpinctax_table(calcX, "weighted_deciles", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"17,004,988","-119,449,251,743","-124,254,447,367","-21,010,701,485"
1,"17,004,489","152,378,742,352","104,382,534,735","-3,341,672,388"
2,"17,006,659","273,930,511,045","175,769,613,967","-1,887,380,105"
3,"17,004,285","400,560,764,101","271,255,931,008","-4,505,952,036"
4,"17,006,306","558,266,448,736","414,206,537,931","-4,527,702,496"
5,"17,005,711","754,024,219,820","606,882,342,664","-6,706,738,832"
6,"17,005,578","1,008,752,286,904","867,578,622,129","-8,918,376,001"
7,"17,004,128","1,379,302,477,096","1,261,792,218,778","-9,160,920,710"
8,"17,006,254","2,054,398,833,867","1,912,589,026,337","-8,773,837,161"
9,"17,006,552","6,272,407,565,304","5,953,769,770,507","168,833,281,214"


In [12]:
create_corpinctax_table(calcX, "weighted_deciles", "weighted_avg")

,expanded_income,s006,c00100,share_corptax_burden
0,"-7,024","17,004,988","-7,307","-1,236"
1,"8,961","17,004,489","6,139",-197
2,"16,107","17,006,659","10,335",-111
3,"23,556","17,004,285","15,952",-265
4,"32,827","17,006,306","24,356",-266
5,"44,339","17,005,711","35,687",-394
6,"59,319","17,005,578","51,017",-524
7,"81,116","17,004,128","74,205",-539
8,"120,803","17,006,254","112,464",-516
9,"368,823","17,006,552","350,087","9,928"


In [13]:
create_corpinctax_table(calcX, "small_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"11,933,771","30,166,679,619","25,766,280,670","-2,785,768,489"
2,"11,222,794","84,841,559,555","63,763,953,735","-2,939,075,734"
3,"11,842,787","149,302,233,838","89,857,596,332","-1,384,770,753"
4,"12,624,678","220,790,072,087","147,819,537,147","-1,405,632,040"
5,"10,000,060","223,259,725,713","151,035,592,537","-2,299,522,141"
6,"8,942,973","246,732,831,091","170,444,698,692","-3,045,664,335"
7,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
8,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
9,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"


In [14]:
create_corpinctax_table(calcX, "large_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"23,156,565","115,008,239,174","89,530,234,405","-5,724,844,223"
2,"24,467,465","370,092,305,925","237,677,133,479","-2,790,402,792"
3,"18,943,034","469,992,556,804","321,480,291,229","-5,345,186,476"
4,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
5,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
6,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"
7,"13,547,772","1,163,806,445,651","1,070,601,241,917","-8,671,679,461"
8,"23,023,733","3,181,461,751,494","2,964,155,988,738","-10,073,393,879"
9,"9,736,321","5,022,083,057,576","4,787,638,191,106","171,433,989,197"


In [15]:
create_corpinctax_table(calcX, "webapp_income_bins", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"4,766,539","-151,159,828,775","-151,251,969,357","-18,064,503,231"
1,"23,156,565","115,008,239,174","89,530,234,405","-5,724,844,223"
2,"24,467,465","370,092,305,925","237,677,133,479","-2,790,402,792"
3,"18,943,034","469,992,556,804","321,480,291,229","-5,345,186,476"
4,"16,371,822","566,134,757,582","423,732,642,474","-4,626,417,503"
5,"13,283,431","596,999,684,070","484,254,671,843","-4,445,028,427"
6,"22,758,269","1,400,153,627,981","1,216,153,724,855","-11,692,533,205"
7,"13,547,772","1,163,806,445,651","1,070,601,241,917","-8,671,679,461"
8,"23,023,733","3,181,461,751,494","2,964,155,988,738","-10,073,393,879"
9,"7,922,248","2,275,472,665,060","2,147,876,336,172","5,829,596,514"


In [16]:
create_diagnostic_table(calcX)

,2017
Returns (#m),170.1
AGI ($b),"11,444.0"
Itemizers (#m),44.6
Itemized Deduction ($b),"1,269.7"
Standard Deduction Filers (#m),111.9
Standard Deduction ($b),985.7
Personal Exemption ($b),"1,173.7"
Taxable Income ($b),"8,500.8"
Regular Tax ($b),"1,777.2"
AMT Income ($b),"10,826.5"


In [17]:
create_diagnostic_table(calcY)

,2017
Returns (#m),170.1
AGI ($b),"11,444.0"
Itemizers (#m),44.6
Itemized Deduction ($b),"1,269.7"
Standard Deduction Filers (#m),111.9
Standard Deduction ($b),985.7
Personal Exemption ($b),"1,173.7"
Taxable Income ($b),"8,500.8"
Regular Tax ($b),"1,835.5"
AMT Income ($b),"10,826.5"


In [18]:
create_corpinctax_table(calcY, "weighted_deciles", "weighted_sum")

,s006,expanded_income,c00100,share_corptax_burden
0,"17,004,988","-119,449,251,743","-124,254,447,367","-21,010,701,485"
1,"17,004,489","152,378,742,352","104,382,534,735","-3,341,672,388"
2,"17,006,659","273,930,511,045","175,769,613,967","-1,887,380,105"
3,"17,004,285","400,560,764,101","271,255,931,008","-4,505,952,036"
4,"17,006,306","558,266,448,736","414,206,537,931","-4,527,702,496"
5,"17,005,711","754,024,219,820","606,882,342,664","-6,706,738,832"
6,"17,005,578","1,008,752,286,904","867,578,622,129","-8,918,376,001"
7,"17,004,128","1,379,302,477,096","1,261,792,218,778","-9,160,920,710"
8,"17,006,254","2,054,398,833,867","1,912,589,026,337","-8,773,837,161"
9,"17,006,552","6,272,407,565,304","5,953,769,770,507","168,833,281,214"


In [22]:
create_distribution_table(calcX.records, groupby ='webapp_income_bins', result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"4,766,539","-151,251,969,357","18,346","35,970,353,999",725,"363,975,419","28,780,402,750","564,689,704","102,098,784","-158,487,319,034",0,0,"102,098,784","6,959,992","100,048,927","740,769,917","-545,582,198","2,698,813,351","2,153,231,153"
1,"23,156,565","89,530,234,405","21,111,011","145,940,013,465","21,010","138,197,975","92,761,892,793","3,245,425,065","246,454,537","89,295,939,298","6,064","2,190,977","248,645,514","1,585,661","55,659,675","8,583,141,657","-8,280,422,130","12,385,822,810","4,105,400,680"
2,"24,467,465","237,677,133,479","19,343,849","189,036,512,602","394,994","4,416,787,617","154,484,145,835","35,730,415,436","3,504,731,923","234,648,212,832","44,211","38,136,204","3,542,868,127","443,195,849","170,126,938","36,409,804,284","-33,140,005,068","33,046,750,389","-93,254,679"
3,"18,943,034","321,480,291,229","16,568,273","152,703,860,777","930,765","13,280,464,516","131,651,369,104","96,914,639,715","10,563,476,356","313,179,231,711","52,624","51,378,121","10,614,854,477","2,213,635,116","271,191,074","25,320,140,420","-16,647,729,985","42,148,407,068","25,500,677,083"
4,"16,371,822","423,732,642,474","14,105,596","132,777,370,133","1,831,280","27,589,523,610","120,048,264,475","183,560,879,579","21,475,223,857","406,344,870,876","18,818","57,367,685","21,532,591,542","4,600,592,758","311,878,860","14,510,123,915","2,733,753,729","52,805,110,762","55,538,864,491"
5,"13,283,431","484,254,671,843","10,747,738","103,281,104,551","2,465,042","39,527,444,053","98,106,689,376","261,735,433,258","32,447,827,605","460,823,053,615","15,455","32,341,984","32,480,169,589","5,539,310,616","387,319,182","5,136,267,301","22,191,910,854","58,122,884,258","80,314,795,112"
6,"22,758,269","1,216,153,724,855","15,425,507","160,802,091,824","7,298,846","133,278,065,344","179,871,956,266","752,556,314,429","102,771,654,403","1,138,015,339,506","29,385","57,448,635","102,829,103,038","12,380,772,584","1,016,885,869","2,726,960,893","88,738,255,430","136,588,157,520","225,326,412,950"
7,"13,547,772","1,070,601,241,917","7,136,642","80,526,049,091","6,406,265","129,123,520,606","118,567,467,009","743,619,635,519","110,910,854,267","999,590,292,933","72,559","162,855,699","111,073,709,966","9,091,262,084","961,869,052","980,731,270","101,963,585,664","121,147,208,933","223,110,794,597"
8,"23,023,733","2,964,155,988,738","6,620,704","80,720,352,310","16,398,723","413,364,507,906","243,905,602,355","2,227,781,900,658","380,917,596,198","2,750,062,272,529","662,137","1,615,496,198","382,533,092,396","16,385,195,867","2,930,825,783","2,287,619,155","366,791,103,157","361,382,032,033","728,173,135,190"
9,"7,922,248","2,147,876,336,172","647,430","7,896,308,336","7,270,201","305,612,624,281","80,479,768,600","1,755,000,559,858","387,714,954,223","2,010,015,982,539","4,132,359","21,532,509,835","409,247,464,058","5,174,754,120","4,957,589,029","2,887,193","409,027,411,774","201,411,055,665","610,438,467,439"


In [21]:
create_distribution_table(calcX.records, groupby ='large_income_bins', result_type='weighted_sum')

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"4,766,539","-151,251,969,357","18,346","35,970,353,999",725,"363,975,419","28,780,402,750","564,689,704","102,098,784","-158,487,319,034",0,0,"102,098,784","6,959,992","100,048,927","740,769,917","-545,582,198","2,698,813,351","2,153,231,153"
1,"23,156,565","89,530,234,405","21,111,011","145,940,013,465","21,010","138,197,975","92,761,892,793","3,245,425,065","246,454,537","89,295,939,298","6,064","2,190,977","248,645,514","1,585,661","55,659,675","8,583,141,657","-8,280,422,130","12,385,822,810","4,105,400,680"
2,"24,467,465","237,677,133,479","19,343,849","189,036,512,602","394,994","4,416,787,617","154,484,145,835","35,730,415,436","3,504,731,923","234,648,212,832","44,211","38,136,204","3,542,868,127","443,195,849","170,126,938","36,409,804,284","-33,140,005,068","33,046,750,389","-93,254,679"
3,"18,943,034","321,480,291,229","16,568,273","152,703,860,777","930,765","13,280,464,516","131,651,369,104","96,914,639,715","10,563,476,356","313,179,231,711","52,624","51,378,121","10,614,854,477","2,213,635,116","271,191,074","25,320,140,420","-16,647,729,985","42,148,407,068","25,500,677,083"
4,"16,371,822","423,732,642,474","14,105,596","132,777,370,133","1,831,280","27,589,523,610","120,048,264,475","183,560,879,579","21,475,223,857","406,344,870,876","18,818","57,367,685","21,532,591,542","4,600,592,758","311,878,860","14,510,123,915","2,733,753,729","52,805,110,762","55,538,864,491"
5,"13,283,431","484,254,671,843","10,747,738","103,281,104,551","2,465,042","39,527,444,053","98,106,689,376","261,735,433,258","32,447,827,605","460,823,053,615","15,455","32,341,984","32,480,169,589","5,539,310,616","387,319,182","5,136,267,301","22,191,910,854","58,122,884,258","80,314,795,112"
6,"22,758,269","1,216,153,724,855","15,425,507","160,802,091,824","7,298,846","133,278,065,344","179,871,956,266","752,556,314,429","102,771,654,403","1,138,015,339,506","29,385","57,448,635","102,829,103,038","12,380,772,584","1,016,885,869","2,726,960,893","88,738,255,430","136,588,157,520","225,326,412,950"
7,"13,547,772","1,070,601,241,917","7,136,642","80,526,049,091","6,406,265","129,123,520,606","118,567,467,009","743,619,635,519","110,910,854,267","999,590,292,933","72,559","162,855,699","111,073,709,966","9,091,262,084","961,869,052","980,731,270","101,963,585,664","121,147,208,933","223,110,794,597"
8,"23,023,733","2,964,155,988,738","6,620,704","80,720,352,310","16,398,723","413,364,507,906","243,905,602,355","2,227,781,900,658","380,917,596,198","2,750,062,272,529","662,137","1,615,496,198","382,533,092,396","16,385,195,867","2,930,825,783","2,287,619,155","366,791,103,157","361,382,032,033","728,173,135,190"
9,"9,736,321","4,787,638,191,106","836,544","10,177,739,710","8,890,348","508,565,493,503","80,615,582,655","4,195,103,770,877","1,114,300,630,539","4,593,051,457,066","4,832,075","43,800,095,954","1,158,100,726,493","24,206,720,650","42,298,906,804","3,536,468","1,176,189,376,180","272,309,947,853","1,448,499,324,033"
